
# Handwritten Digit Recognition (MNIST Dataset)
<hr>

![MNIST](https://neurohive.io/wp-content/uploads/2019/05/Screenshot-from-2019-05-29-21-23-47.png)
### History of Handwritten Digit dataset 

Modified National Institute of Standards and Technology database (MNIST dataset) is a large dataset of handwritten digits which is widely used in image processing and machine learning. The set of images in the MNIST database is a combination of two of NIST's databases: Special Database 1 and Special Database 3. Special Database 1 and Special Database 3 consist of digits written by high school students and employees of the United States Census Bureau, respectively.<br> 


[READ MORE](https://en.wikipedia.org/wiki/MNIST_database)

### Why to start with MNIST dataset?
The MNIST dataset is a well-known dataset for image classification. Tensorflow and Keras also provide MNIST dataset directly through their APIs. For the learning purpose, the MNIST dataset is easy to use and experiment with different machine learning techniques.

### About the Notebook
* In this notebook, I have covered the necessary steps to approach any Machine Learning Classification Problem.
* Included Image Visualization for better understanding.
* Quick Links to the functions I have used to explore it in depth.
* Basic techniques such as Confusion Matrix, Image Augmentation, etc.
* I have also compared the results of Model without using CNN and CNN.

I have tried to make this notebook as simple as possible, along with covering the basic approach to tackle any classification task. 
### Task
The task is to classify the images in 10 class, i.e., [0-9], inclusively.  
<hr>

<font style="color:Red;font-size:18px">If you find this notebook helpful, Please UPVOTE.</font>

### Follow me:

* <a href="https://bit.ly/tarungithub"> GitHub</a>
* <a href="https://bit.ly/tarnkr-youtube">Youtube<a/>
* <a href="https://medium.com/@codeeasy"> Medium</a>
* <a href="https://www.linkedin.com/in/tarun-kumar-iit-ism/">Linkedin</a>



<hr>

# Content:

* [Handwritten Digit Recognition (MNIST Dataset)](#Handwritten-Digit-Recognition-(MNIST-Dataset))
* [Required Imports](#Required-Imports)
* [Loading and Visualizing Dataset](#Loading-and-Visualizing-Dataset)
* [Visualize Digits dataset](#Visualize-Digits-dataset)
* **[Buliding Model](#Buliding-Model)**
    * [Model Using Keras](#Model-Using-Keras)
    * [Compiling Model](#Compiling-Model)
    * [Training](#Training)
    * [Training Performance](#Training-Performance.)
    * [Confusion Matrix](#Confusion-Matrix)
* **[Improving Result by Image Augmentation](#Improving-Result-by-Image-Augmentation)**
    * [Augmentation Using Keras](#Augmentation-Using-Keras)
    * [Learning Rate](#Learning-Rate)
* [Visualizing Result](#Visualizing-Result)
* [Predict on Testset](#Predict-on-Testset)
* **[Understand the Intermediate Layers of the Model](#Lets-understand-the-intermediate-layers-of-the-model)**
    * [Compare the layer 1 output](#Try-to-compare-the-layer-1-output)
    * [Visualizing All Intermediate Activation Layer](#Visualizing-All-Intermediate-Activation-Layer)

# Required Imports

**Imports:**
* pandas : For handeling csv dataset
* numpy : Support for Pandas and calculations
* Matplotlib - For visualization (Plotting graphs)
*  keras - Prediction Models

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import cv2 as cv

from keras.layers import Conv2D, Input, LeakyReLU, Dense, Activation, Flatten, Dropout, MaxPool2D
from keras import models
from keras.optimizers import Adam,RMSprop 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

import pickle

%matplotlib inline

# Loading and Visualizing Dataset
<hr>

## About Dataset
MNIST dataset has the following features:
* Dataset size 60,000 samples of handwritten images.
* The size of each image is 28x28 pixels.
* Each image has only 1 color channel, i.e., grayscale image.
* Each pixel has value in the range of [0,255] where 0 represents black, and 255 represents white.
* Each image has labeled from 0-9.


## Loading train.csv

This will loads the data from Kaggle dataset train.csv into a **Dataframe**. As file type is CSV, I am loading it using [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) of **pandas**. Then using NumPy for **random permutation** of the training dataset. I am using seed to regenerate the same permutation every time. Change seed value to get different permutation. 
<br>

**Check out these functions for more info:**
* [df.iloc[]](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html)
* [np.random](https://docs.scipy.org/doc/numpy-1.14.0/reference/routines.random.html)

In [ ]:
np.random.seed(1) # seed
df_train = pd.read_csv("../input/digit-recognizer/train.csv") 
# Loading Dataset
df_train

In [ ]:
df_train = df_train.iloc[np.random.permutation(len(df_train))] 
# Random permutaion for dataset (seed is used to resample the same permutation every time)
# 数据集的随机排列,np.random.permutation生成随机序列
df_train

In [ ]:
np.random.permutation(len(df_train))

In [ ]:
df_train.head(5)

In [ ]:
df_train.shape

Training set has 42,000 images. And has 785 columns, 1st coloumn is label for the image and rest 784 are the pixel values. <br>**Remember it is flattened. I will reshape it latter.**

### Preparing Training and Validation data 
**It requires a few steps:**
* Assuming the validation set size. I am taking it 10% of the training set.  
* Splitting training set into a training set (90% original training set) and validation set (10% original training set) from the training dataset.
* Reshaping both sets into (sample size,28,28,1) where sample size represents the size of the train or validation set.
* Splitting the labels for both training set and validation set.

Using df.iloc of pandas for slicing the data frame(read more link below), then converting into NumPy array and finally reshaping NumPy array in required shape.<br>

**Check out these functions for more info:**
* [df.iloc[]](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html)
* [np.reshape](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html)
* [np.asarray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.asarray.html)

In [ ]:
sample_size = df_train.shape[0] # Training set size
validation_size = int(df_train.shape[0]*0.1) # Validation set size 

# train_x and train_y
train_x = np.asarray(df_train.iloc[:sample_size-validation_size,1:]).reshape([sample_size-validation_size,28,28,1]) 
# taking all columns expect column 0
train_y = np.asarray(df_train.iloc[:sample_size-validation_size,0]).reshape([sample_size-validation_size,1]) 
# taking column 0

# val_x and val_y
val_x = np.asarray(df_train.iloc[sample_size-validation_size:,1:]).reshape([validation_size,28,28,1])
val_y = np.asarray(df_train.iloc[sample_size-validation_size:,0]).reshape([validation_size,1])

#### Shape of training set

In [ ]:
train_x.shape,train_y.shape

## Loading test.csv
This will load the test.csv. It has 18,000 images and no label is there. Prediction need to be done on these images. After loading converting the data in form of numpy array and reshaping it.

In [ ]:
df_test = pd.read_csv("../input/digit-recognizer/test.csv")
test_x = np.asarray(df_test.iloc[:,:]).reshape([-1,28,28,1])

In [ ]:
test_x.shape

In [ ]:
np.max(test_x)

## Normalize Pixel Data
Each pixel values lies between [0,255]. This value range is too high and it will be difficult for any model to learn. The best approach is **normalize** the data. In this case, as the pixel value is in the known range it sufficient to **scale the pixel values** in range [0,1] by simply dividing the array by **255**. 

In [ ]:
# convirting pixel values in range [0,1]
train_x = train_x/255
val_x = val_x/255
test_x = test_x/255

## Visualize Digits dataset
<hr>

The first and fundamental thing to check is the frequency of the classes in the dataset, as the balanced dataset is always good to start. But this is not always true, and there are several supervised learning tasks in which the classes are not balanced, also in case of anomaly detection, there is a large difference between the positive and negative class. But that is out of the scope of this notebook.

### 1. Frequency plot for the training set

In [ ]:
df_train.iloc[:sample_size-validation_size,:].groupby('label')['label'].count()

In [ ]:
# Cheacking frequency of digits in training and validation set
counts = df_train.iloc[:sample_size-validation_size,:].groupby('label')['label'].count()
# df_train.head(2)
# counts
f = plt.figure(figsize=(10,6))
f.add_subplot(111)

plt.bar(counts.index,counts.values,width = 0.8,color="orange")
for i in counts.index:
    plt.text(i,counts.values[i]+50,str(counts.values[i]),horizontalalignment='center',fontsize=14)

plt.tick_params(labelsize = 14)
plt.xticks(counts.index)
plt.xlabel("Digits",fontsize=16)
plt.ylabel("Frequency",fontsize=16)
plt.title("Frequency Graph training set",fontsize=20)
plt.savefig('digit_frequency_train.png')  
plt.show()

In [ ]:
counts[1]

### 2. Frequency plot for the validation set

In [ ]:
# df_train.iloc[sample_size-validation_index:,1:]
# Cheacking frequency of digits in training and validation set
counts = df_train.iloc[sample_size-validation_size:,:].groupby('label')['label'].count()
# df_train.head(2)
# counts
f = plt.figure(figsize=(10,6))
f.add_subplot(111)

plt.bar(counts.index,counts.values,width = 0.8,color="orange")
for i in counts.index:
    plt.text(i,counts.values[i]+5,str(counts.values[i]),horizontalalignment='center',fontsize=14)

plt.tick_params(labelsize = 14)
plt.xticks(counts.index)
plt.xlabel("Digits",fontsize=16)
plt.ylabel("Frequency",fontsize=16)
plt.title("Frequency Graph Validation set",fontsize=20)
plt.savefig('digit_frequency_val.png')
plt.show()

It seems that both the training and validation set has a good balance between the classes, so let's move on and see a few of the digits.

### Visualizing the digits by plotting Images
<hr>
This will plot the first 30 images of digits with the label.

In [ ]:
rows = 5 # defining no. of rows in figure
cols = 6 # defining no. of colums in figure

f = plt.figure(figsize=(2*cols,2*rows)) # defining a figure 
# 在matplotlib一般使用plt.figure来设置窗口尺寸。
for i in range(rows*cols): 
    f.add_subplot(rows,cols,i+1) 
    # adding sub plot to figure on each iteration
    plt.imshow(train_x[i].reshape([28,28]),cmap="Blues") 
    plt.axis("off")
    plt.title(str(train_y[i]), y=-0.15,color="green")
plt.savefig("digits.png")

In [ ]:
plt.imshow(train_x[1].reshape([28,28]))

In [ ]:
plt.imshow(train_x[1].reshape([28,28]),cmap="Blues")

In [ ]:
rows = 5 # defining no. of rows in figure
cols = 1 # defining no. of colums in figure

f = plt.figure(figsize=(2^cols,2*rows)) # defining a figure 
# f = plt.figure(figsize=(2,2))
for i in range(rows*cols): 
    f.add_subplot(rows,cols,i+1) 
    # adding sub plot to figure on each iteration
    plt.imshow(train_x[i].reshape([28,28]),cmap="Blues") 
    plt.axis("off")
    plt.title(str(train_y[i]), y=-0.15,color="green")
plt.savefig("digits.png")

# Buliding Model

<hr>

![model](https://miro.medium.com/max/1000/1*4OUonEDfZwCfR4Y-G-h1fw.jpeg)

In [ ]:
# # Loading pickled resources
# !wget https://www.kaggleusercontent.com/kf/31703703/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..f6l0FhjgIxCd7bfZA3MF_A.oHtR56AzfPZhslGD-R2Uca6Kce1yVCG805lWUk25H5T-MQciLjAjmoTd9RPGh1zdUlwtbMZPuAi9_1BrHNZfFlJ5duoYajHON-Sk_mMy7OIePjqNRqo8vkEnTHmbV6Oj1Z6MR9dGzT2Gch2soeaLaZnIjxJt5e8DsFaia6dTjxRzzzrKaQDWLikdsjo2xwbQp9yo4-8htw6adclSbtnXsMV4kJNBs25d-qqRLuUSuhqxxCbJMkuuZgPjnEzqO7aLU0zqcGYUXDDdx1O-oU2ncMAYpXYqssqzQgD6-t4Fl83XWQnNqRv5wec5cdD-7IF9cbjyD_CE-Ib863pPJ9RJc-IYypbUvvKfMQuhahe9NiuRGNSNodVlSiuSzk0nudl5uHqf7V7_1h_juPPVj8mUUOqleLye9_ZtJ2S8pD6hUXT9p7kPy6v6RdoaE_LgkrijyvmJhmS-yMETpazrlQlKp96A3W0EVdhtVxmW7QUwbjIlzdEs7whAe4EcqQIzd4H69TR6hLCqVlaZkMBBPvBWr_dCTxu6htDP8qE2QCH08H1VPXyZLERTMH1SRENnwa_BxMTVkc_pP70tkvGA2xtgoJHzAlcOZZfqsa5fmCa8tqIOkME1hn88Xgm5eh58JXT2ZXyA7hxfpzKP3UQXegeBnPaEPOLa5eEoND9E9ypyi5I1QWa7QTMe9ruEbXr6DUbJ.GvQemyhjKibaOnfNrnRUXQ/model.h5
# !wget https://www.kaggleusercontent.com/kf/31703703/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..yUikFJ_W78H0Zfo7LsQ-2g.1-XOTVZwAUnN9uzbSyaPEuXbdS_5mlLvYRUOaFjMZewxYpcCFASz2fROWEIHjC-KHCOOO-DYjLY7pWHvN81cOW25n28C2mT6aFnWkObrkUYXlbZW0sS7iaKwxUw4a_XwYYUmMfeNuOpKf5OsBdv-L2y0GOLb-fT8hDqteihh6qP97VOT1fmvdv2gYcG0WqKw4mcWwtWAYyqQAScE_knALnyTvWZTF73LR99gaxy_w41t3PHo0rfHy037Cll0lWznkf6ppfAIv_CKr_v2HfpB9go0DJbXKqD87LkcnNF94e_b95i0UYfS2pMugXS4ob1WnSblE34Df_n9rB3pWMXn9DPnqlwDqa1snJOc3CLeK9MTsPQ9NuzEFIjxMnWXQhNGQ7sXYX8qAPqanQ3-OQSZHHD8lufpcJEguVqOuOWY5qZlKe4ZTzn3g8fhbssulphnajM4ZThC5pceWnVT2rowvERPtlvijy6AdAGEy57BxN7FeweBVZAR1Zv5RA_wE2qo4DFmyK36j8p_bVZHTnVvqFFWz0SAimJqzMmzxNrZY7_NGuqDy5rjUmoa2y0e0-qPFBcfFWvT2Pe_RKW21dBOKH9HbI1j0WS_Ua72FI1-MToX8DUHRdN9UNLJhnugwWY_lWvppU-fRCJFhkSPIYyfzbx1cH2ykLLZRz4-sjCegE78I03ue3096Q7kUMw9-ZGd.a3Ge-GJWsUMNPuyQ0gpMfg/model_img_augmentation.h5
# !wget https://www.kaggleusercontent.com/kf/31703703/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..Hq9sgF7KDfRSLRlO3zlQSg.dQKW1YCUjhwAZbX-54g1ygj0qTWSH6egywwOdIqm-hwUxTWnj6L9EkNK8qncdH4FeNbEtAjYrQ2oStYGIVVJzsqFDQwaXBsnzvKQx2icckW3ww-aNtDxATdkVY2ZiqMLAMIMyy8_nFoFvt2tQm48XeIUecn3OTL6Te3VfRr3OXQ1QWU7lbY-8BetDVy0tbrLV_vIkV-fUy_FLGsa7QgH4Xerxi7xXoItAmJCbEIXBt5pR-_frNz0rdDoj30e-xGdUkLRiBNe1Nk9_1EFacGwTwhM3KqE8SDF5CtVqP7XsFhFIxVal-lmUy1hzsPi0xZt_ikRIbmfTb6K5HmQ97Jzm4nd7YdQ_u2ScbhLLFy2pj0n4XapGZgMYE5o_2_Cv2c_3uquTGDTpjiOAA25ylrFypwZGmenAeCSrJZIto0ta_onqVPz_euNQY4PHJ5P3aJi2KWHJzNl3LxBU2u-LfkfOzwYXu1DIOOqkzSSLWraxZoleean-9oawdqq0doQhogfx2wBr844ApCpVYDxEEn8CxAwfK-RoiE7MdEasDYtWujlmcOnMQsQjUohnJcOWwQSIOIEpqevcXBU4PPoiw-ex-vEUtflupnNRtlwwLRzjyrV_6VJbdQkus1F0HfToHEBjB6tkZyydreIK3Zi8u0l1YvT3VRtWMw3amc3s7f86ilZZ_uoGhW8fc7k14JmubAX.FiOfKdbaYf0TStGFpmhDSw/history_1.hs
# !wget https://www.kaggleusercontent.com/kf/31703703/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..IWrH9XKSk0oDOCNg7wTlbg.A49rZubI479_e2da4l7JhZBGm7YdJ87UviDnyAP6924Kez0dRIPjzw-67wCI_iMtJGQQJrnn74pVwzt0cPa8CVTJIApHs2t4qZd-7dlWDcOqcwlC8zaLECXT836Jey6auUY1JE1C8YApSHBfDr-WB_KO75JQRzDdXV6LFEdPfzTyoXWf0zmuyg4bi41rPhH2UkhEfPowmze9G_nOWg64WHfJTBCcWzMDf56GbyNmp46RtdKuqriMH0sAHHFrz94DQkHnnz0U149yYC6oUkqlvty-l7jvEGM8Nc1_-LZbptB_8cZxu9gOFYprjdUyAerb5Izz-J1nkextV-0OUe8SUYj8XwG5pF6pNDbZrkoDTurkkORlMIEAKNhl6SZvPjmb1Mv-owxnrpSdHnHejp61kFp-FOi_oFFzAJiW5NVm8pmMln7RP3JgcuCfWKQUEHlrsC5NaMvsFG9CNlXeul9E4PXS6ycuRzflz5uWxvoe1Dt-Tm4Pnoaa-CvJ2hsLKsX-oH66LE6sWQm8UrDTJ0eBxaWG8QAbJ6WeOICv9zykRx8ro5j18RWgnOLK2x9PLJmYA7rL6Zm7kxavLQMTovSg-IZ9EyuTuSmGv-0Rd_0Gmb5-2qvBsYKGcn8lbJMmOXfeDWEqTT0XBK-VfVcDNxX8EZd2iYD1c3nJa6R324v4JGTOLHZJbL7UNtGXtQn-3WVc.ShxYOf_zson_2CAlaDOT8w/history_2.hs
!wget https://www.kaggleusercontent.com/kf/32045042/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..LcCpstRicdTOqMGGjh1wmg.LJi-FKBMbLQL5cc2XHtVyNpT--iVgBeKlbby9C_YjBjm7dSLbbqY4F27EpcgOWZYo22EOWWwyVMAd7VsYOzQkUGK-sj6gvWswv9CTc8I8ZDMkZvGdRRy3COevzGk21yXTlbRG1D--BVHlwCZVaYSMBKt9gIeCELgl-ZcFKoOjZSUpcZXBrmfuBA_OK63fc_olldErb5p5S_qWVecdJ1r49anGVs-x682Q0y5cCs9lr7q1n2B2saWkiC4d8tMGRMyUTCt1xXl6nENaYNh4C4u0DGOBfT14jEuWfyzFH3obgs_TtD4Bqg2zzxjjUwtJd7ymBNCVq7td-_chLlds-lps5uq4BLckniKl9QgbJ7ZcL6qoMonLzPn5VUE-vlIfUCfBieYOHKkWb3Buz557IrG4mk9GqAvOPp_Vzd5n9h9SGEvOFCsiG-_IinSIA7a2NxSPvtNEKmYcG37CS0xLfEV8copUh3pIHdVVLn9SXd2r5Wg2Q5q8zUnEvnHSxcPNg7LheviS4NMX18bsL9Wqm2PP_WDd7QkO_wJ0AsGyDveLWKebHB26wS_iz3w7X-EAheRmBrCyiUfJWHSM-IJPAJk9EIhRH1zr8WVtwsc19PRevFcY4RDHx7fyvPFVrJ2fFsicZMaDfJchFR_lP_uYDs6NvPu5AXXEkXnFMsrJ0IAICiEDwadnj0eV5w5b-DozhrP.8pmfOFY56dYt_Z2UVg-UWA/model_img_augmentation.h5

### Model Using Keras

There are two different ways of defining the Model in Keras:
* Sequential Model
* Function API

Functional API is used to build a more complicated Model such as for multi-output Models, directed acyclic graphs, or models with shared layers. I am using the Sequential Model in this notebook to keep things simple. 
<br>
In Sequential Model, you can add each layer sequentially.<br>

**Description of Model:**
* 2 Convolutional Blocks
    > Each block consists of 2 Conv2D layers with LeakyRelU activation layers. Then a MaxPool2D layer and finally a Dropout Layer. 
* Then Dense Layers and Output layer after Flatten layer.
* MaxPool2D layer is used to reduce the size of the image. Pool size `(2,2)` means reducing the image from `(28,28)` to `(14,14)`. Reducing the features.
* Dropout layer drops the few activation nodes while training, which acts as regularization. Do let the model to over-fit.
* Output layer has 10 nodes with sigmoid activation.

**Check out these functions for more info:**
* [Conv2D](https://keras.io/layers/convolutional/#conv2d)
* [LeakyReLU](https://keras.io/layers/advanced-activations/#leakyrelu)
* [MaxPool2D](https://keras.io/layers/pooling/#maxpool2d)
* [Dropout](https://keras.io/layers/core/#dropout)
* [Flatten](https://keras.io/layers/core/#flatten)
* [Dense](https://keras.io/layers/core/#dense)

In [ ]:
model = models.Sequential()

In [ ]:
# Block 1
model.add(Conv2D(32,3, padding  ="same",input_shape=(28,28,1)))
# filter数目：32
# filter大小：3*3
model.add(LeakyReLU())
model.add(Conv2D(32,3, padding  ="same"))
model.add(LeakyReLU())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# output：14*14*32

# Block 2
model.add(Conv2D(64,3, padding  ="same"))
model.add(LeakyReLU())
model.add(Conv2D(64,3, padding  ="same"))
model.add(LeakyReLU())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# output：7*7*64

model.add(Flatten())
# output：7*7*64=3136

model.add(Dense(256,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(10,activation="sigmoid"))

### Compiling Model
Model compilation required the selection of optimizer and loss function. Let me discuss a few important things to avoid confusion.

**Optimizers:** Keras provides several optimizers that can be used by importing the optimizers and passing in compile function.
* SGD (Stochastic gradient descent optimizer)
* RMSprop 
* Adam
* Adamax

There are several more check it out [here](https://keras.io/optimizers/). 
<br>

One of the important things is the selection of the **learning rate**. If the learning rate is too high, the loss may not converge, and if it is too low, the training will be slow. So it is important to select the reasonably fair value of learning rate. One of the good value to start with is 0.001. If it doesn't work, then other higher or lower values can be tried. The rest of the parameters generally works well and need not be defined. The default value works most of the time.
Here, I am using **Adam optimizer**, but **RMSprop** can also be used.
<br>

**Loss Functions:** Keras provides all of the well-known loss functions which work well for most of the time. But if you need to define a custom loss function, you can. Defining a custom function is out of the scope of this notebook. Let's understand the loss function for the classification tasks. I am discussing 3 loss function here. There are several more check it out [here](https://keras.io/losses/). 
* **binary_crossentropy**: This loss function is used for the binary classification task. The single-node output layer is required. 0 and 1 is used for classification.
* **categorical_crossentropy**:  Used for Used for Multi-class classification 
* **sparse_categorical_crossentropy:** Used for Multi-class classification. <br>

**Difference between categorical_crossentropy and sparse_categorical_crossentropy:**
* If your targets are one-hot encoded, use categorical_crossentropy.
    Examples of one-hot encodings (for three classes):
    * [1,0,0]
    * [0,1,0]
    * [0,0,1]
* But if your targets are integers, use sparse_categorical_crossentropy.
    Examples of integer encodings (for the sake of completion):
    * 1
    * 2
    * 3
    
Here, I am using **sparse_categorical_crossentropy**, as the target values are integer not **one-hot vector**.


In [ ]:
initial_lr = 0.001
loss = "sparse_categorical_crossentropy"
model.compile(Adam(lr=initial_lr), loss=loss ,metrics=['accuracy'])
model.summary()
# optimizer: 优化器
# loss:损失函数,可以用自带的,也可以自定义.如果模型有多个输出,
#     可以传入一个字典或者损失列表,模型降会把这些损失加在一起
# metrics: 列表，包含评估模型在训练和测试时的性能的指标，典型用法是metrics=[‘accuracy’]。

### Training 
[model.fit()](https://keras.io/models/sequential/#fit) is used to train the model. It takes training data, batch_size, no of epochs, validation data. There are several more parameters, and you can check the documentation [here](https://keras.io/models/sequential/#fit).
I am taking the epochs = 20 and batch size = 256. It will return the history of training, which later can be used to analyze the performance.

In [ ]:
epochs = 20
batch_size = 256
history_1 = model.fit(train_x,train_y,batch_size=batch_size,epochs=epochs,validation_data=[val_x,val_y])

In [ ]:
history_1.epoch

In [ ]:
history_1.history

In [ ]:
model.save("model.h5")
with open('history_1.hs', 'wb') as history:
    pickle.dump(history_1,history)
    
# pickle.dump(obj, file, [,protocol])
# 函数的功能：将obj对象序列化存入已经打开的file中。

# pickle.load(file)
# 函数的功能：将file中的对象序列化读出

In [ ]:
# model = models.load_model("model.h5")
# with open('history_1.hs', 'rb') as history:
#     history_1 = pickle.load(history)

### Training Performance.
Let's see how the training goes—plotting the accuracy and loss of both training and validation set with each epoch. In the accuracy graphs, there is clearly a difference in the training and validation set. The model is more accurate on the training set. It seems that the model is a little bit **overfit**. So can we do better?

In [ ]:
# Diffining Figure
f = plt.figure(figsize=(20,7))

#Adding Subplot 1 (For Accuracy)
f.add_subplot(121)

plt.plot(history_1.epoch,history_1.history['accuracy'],label = "accuracy") 
# Accuracy curve for training set
plt.plot(history_1.epoch,history_1.history['val_accuracy'],label = "val_accuracy") 
# Accuracy curve for validation set

plt.title("Accuracy Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Accuracy",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

#Adding Subplot 1 (For Loss)
f.add_subplot(122)

plt.plot(history_1.epoch,history_1.history['loss'],label="loss") 
# Loss curve for training set
plt.plot(history_1.epoch,history_1.history['val_loss'],label="val_loss")
# Loss curve for validation set

plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

plt.show()

### Confusion Matrix
In the field of **machine learning** and specifically the problem of **statistical classification**, a confusion matrix, also known as an **error matrix**, is a specific table layout that **allows visualization of the performance of an algorithm**, typically a supervised learning one (in unsupervised learning it is usually called a matching matrix). Each row of the matrix represents the instances in a predicted class, while each column represents the instances in an actual class (or vice versa). The name stems from the fact that it makes it easy to see if the system is confusing two classes (i.e. commonly mislabeling one as another). [Source](https://en.wikipedia.org/wiki/Confusion_matrix)

Let's try to see how well the model is performing on the validation set.

In [ ]:
val_p = np.argmax(model.predict(val_x),axis =1)

error = 0
confusion_matrix = np.zeros([10,10])
for i in range(val_x.shape[0]):
    confusion_matrix[val_y[i],val_p[i]] += 1
    if val_y[i]!=val_p[i]:
        error +=1
        
print("Confusion Matrix: \n\n" ,confusion_matrix)
print("\nErrors in validation set: " ,error)
print("\nError Persentage : " ,(error*100)/val_p.shape[0])
print("\nAccuracy : " ,100-(error*100)/val_p.shape[0])
print("\nValidation set Shape :",val_p.shape[0])

In [ ]:
val_p.shape

In [ ]:
f = plt.figure(figsize=(10,8.5))
f.add_subplot(111)

plt.imshow(np.log2(confusion_matrix+1),cmap="Reds")
plt.colorbar()
plt.tick_params(size=5,color="white")
plt.xticks(np.arange(0,10),np.arange(0,10))
plt.yticks(np.arange(0,10),np.arange(0,10))

threshold = confusion_matrix.max()/2 

for i in range(10):
    for j in range(10):
        plt.text(j,i,int(confusion_matrix[i,j]),horizontalalignment="center",color="white" if confusion_matrix[i, j] > threshold else "black")
        
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig("Confusion_matrix1.png")
plt.show()

# Improving Result by Image Augmentation

<hr>

![augmentation](https://cdn-images-1.medium.com/max/1000/1*C8hNiOqur4OJyEZmC7OnzQ.png)

A deep network requires extensive data to achieve decent performance. To build a good classifier with small training data, image augmentation can solve the problem to a greater extend. Image augmentation generates images by different ways of processing, such as random shift, rotation, flips, etc. 
<br>

You can check great python package for data [augmentation](https://github.com/albumentations-team/albumentations).

![Augmentation](https://camo.githubusercontent.com/43d652646b37ef66762212c0e0d3150ba481347c/68747470733a2f2f686162726173746f726167652e6f72672f776562742f73752f77612f6e702f737577616e70656f36777737777077746f6274727a645f636732302e6a706567)*Image [source](https://github.com/albumentations-team/albumentations)* 


### Augmentation Using Keras

<hr>

Here I am using the ImageDataGenerator() function of Keras for Image augmentation. Parameters to use:

* **rotation_range:**   randomly rotate images in the range (degrees, 0 to 180)
* **zoom_range:**  Randomly zoom image 
* **width_shift_range:**  randomly shift images horizontally (fraction of total width)
* **height_shift_range:**   randomly shift images vertically (fraction of total height)
* **horizontal_flip:**   randomly flip images (Can't be used in this case as it changes the digit)
* **vertical_flip:**  randomly flip images (Can't be used in this case as it changes the digit)

Read more about [ImageDataGenerator()](https://keras.io/preprocessing/image/)

After the creation and configuration of the ImageDataGenerator, you must fit it on the data, which calculates any statistics required to perform the transformation on the data. This can be done by calling the fit() function on datagen.

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(train_x)

**ImageDataGenerator()**

* **featurewise_center**：布尔值，使输入数据集去中心化（均值为0）, 按feature执行。

* **samplewise_center**：布尔值，使输入数据的每个样本均值为0。

* **featurewise_std_normalization**：布尔值，将输入除以数据集的标准差以完成标准化, 按feature执行。

* **samplewise_std_normalization**：布尔值，将输入的每个样本除以其自身的标准差。

* **zca_whitening**：布尔值，对输入数据施加ZCA白化。

* **rotation_range**：整数，数据提升时图片随机转动的角度。随机选择图片的角度，是一个0~180的度数，取值为0~180。

* **width_shift_range**：浮点数，图片宽度的某个比例，数据提升时图片随机水平偏移的幅度。

* **height_shift_range**：浮点数，图片高度的某个比例，数据提升时图片随机竖直偏移的幅度。 

* **height_shift_range**和**width_shift_range**是用来指定水平和竖直方向随机移动的程度，这是两个0~1之间的比例。

* **shear_range**：浮点数，剪切强度（逆时针方向的剪切变换角度）。是用来进行剪切变换的程度。

* **zoom_range**：浮点数或形如[lower,upper]的列表，随机缩放的幅度，若为浮点数，则相当于[lower,upper] = [1 - zoom_range, 1+zoom_range]。用来进行随机的放大。

* **channel_shift_range**：浮点数，随机通道偏移的幅度。

* **fill_mode**：‘constant’，‘nearest’，‘reflect’或‘wrap’之一，当进行变换时超出边界的点将根据本参数给定的方法进行处理

* **cval**：浮点数或整数，当fill_mode=constant时，指定要向超出边界的点填充的值。

* **horizontal_flip**：布尔值，进行随机水平翻转。随机的对图片进行水平翻转，这个参数适用于水平翻转不影响图片语义的时候。

* **vertical_flip**：布尔值，进行随机竖直翻转。

* **rescale**: 值将在执行其他处理前乘到整个图像上，我们的图像在RGB通道都是0~255的整数，
这样的操作可能使图像的值过高或过低，所以我们将这个值定为0~1之间的数。

* **preprocessing_function**: 将被应用于每个输入的函数。该函数将在任何其他修改之前运行。该函数接受一个参数，为一张图片（秩为3的numpy array），并且输出一个具有相同shape的numpy array

* **data_format**：字符串，“channel_first”或“channel_last”之一，代表图像的通道维的位置。该参数是Keras 1.x中的

* **image_dim_ordering**，“channel_last”对应原本的“tf”，“channel_first”对应原本的“th”。以128x128的RGB图像为例，
 “channel_first”应将数据组织为（3,128,128），而“channel_last”应将数据组织为（128,128,3）。该参数的默认值是
 ~/.keras/keras.json中设置的值，若从未设置过，则为“channel_last”。

### Learning Rate

<hr>

ReduceLROnPlateau() is a callback function provided by Keras, which is used to reduce the learning rate if when a metric has stopped improving.

<br>

**Parameters:**

* **monitor:**  takes the metric to observe (In this case val_accuracy)
* **patience:** waits for that much epochs for the improvements, if not, then decrease the learning rate. (here `2`)
* **factor:** factor by which the learning rate will be reduced. new_lr = lr * factor (here `0.5`)
* **min_lr:** lower bound on the learning rate. (here `0.00001`)

**Read More:**
[Read more](https://keras.io/callbacks/#reducelronplateau)



In [ ]:
lrr = ReduceLROnPlateau(monitor='val_accuracy',patience=2,verbose=1,factor=0.5, min_lr=0.00001)
# verbose：日志显示
# verbose = 0 为不在标准输出流输出日志信息
# verbose = 1 为输出进度条记录
# verbose = 2 为每个epoch输出一行记录

### Further Traning
[model.fit_generator()](https://keras.io/models/sequential/#fit_generator) is used to train the model on data generated batch-by-batch by image augmentation. The data generator is an iterator that generates and provides data as per request by fit_generator(). We can configure the batch size and get the batches by calling the flow() function.
是用图像增强的方法对逐批生成的数据进行训练。


In [ ]:
epochs = 20
history_2 = model.fit_generator(datagen.flow(train_x,train_y, batch_size=batch_size),steps_per_epoch=int(train_x.shape[0]/batch_size)+1,epochs=epochs,validation_data=[val_x,val_y],callbacks=[lrr])

In [ ]:
model.save("model_img_augmentation.h5")
with open('history_2.hs', 'wb') as history:
    pickle.dump(history_2,history)

In [ ]:
model = models.load_model("model_img_augmentation.h5")
# with open('history_2.hs', 'rb') as history:
#     history_2 = pickle.load(history)

### Training Performance.
Now we can see that after further training, the accuracy of training and validation set almost converges with high accuracy. It seems that the model has been significantly improved after image augmentation.

In [ ]:
# Diffining Figure
f = plt.figure(figsize=(20,7))
f.add_subplot(121)

#Adding Subplot 1 (For Accuracy)
plt.plot(history_1.epoch+list(np.asarray(history_2.epoch) + len(history_1.epoch)),history_1.history['accuracy']+history_2.history['accuracy'],label = "accuracy") # Accuracy curve for training set
plt.plot(history_1.epoch+list(np.asarray(history_2.epoch) + len(history_1.epoch)),history_1.history['val_accuracy']+history_2.history['val_accuracy'],label = "val_accuracy") # Accuracy curve for validation set

plt.title("Accuracy Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Accuracy",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()


#Adding Subplot 1 (For Loss)
f.add_subplot(122)

plt.plot(history_1.epoch+list(np.asarray(history_2.epoch) + len(history_1.epoch)),history_1.history['loss']+history_2.history['loss'],label="loss") # Loss curve for training set
plt.plot(history_1.epoch+list(np.asarray(history_2.epoch) + len(history_1.epoch)),history_1.history['val_loss']+history_2.history['val_loss'],label="val_loss") # Loss curve for validation set

plt.title("Loss Curve",fontsize=18)
plt.xlabel("Epochs",fontsize=15)
plt.ylabel("Loss",fontsize=15)
plt.grid(alpha=0.3)
plt.legend()

plt.show()

## Confusion Matrix

In [ ]:
val_p = np.argmax(model.predict(val_x),axis =1)

error = 0
confusion_matrix = np.zeros([10,10])
for i in range(val_x.shape[0]):
    confusion_matrix[val_y[i],val_p[i]] += 1
    if val_y[i]!=val_p[i]:
        error +=1
        
confusion_matrix,error,(error*100)/val_p.shape[0],100-(error*100)/val_p.shape[0],val_p.shape[0]

print("Confusion Matrix: \n\n" ,confusion_matrix)
print("\nErrors in validation set: " ,error)
print("\nError Persentage : " ,(error*100)/val_p.shape[0])
print("\nAccuracy : " ,100-(error*100)/val_p.shape[0])
print("\nValidation set Shape :",val_p.shape[0])

In [ ]:
f = plt.figure(figsize=(10,8.5))
f.add_subplot(111)

plt.imshow(np.log2(confusion_matrix+1),cmap="Reds")
plt.colorbar()
plt.tick_params(size=5,color="white")
plt.xticks(np.arange(0,10),np.arange(0,10))
plt.yticks(np.arange(0,10),np.arange(0,10))

threshold = confusion_matrix.max()/2 

for i in range(10):
    for j in range(10):
        plt.text(j,i,int(confusion_matrix[i,j]),horizontalalignment="center",color="white" if confusion_matrix[i, j] > threshold else "black")
        
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig("Confusion_matrix2.png")
plt.show()

# Visualizing Result

<hr>

### All Errors in the Validation set
Let's see all the errors in the validation set. It seems that in most of the cases, the recognition of digits is difficult for even humans. So we can say that our model is performing well.

In [ ]:
rows = 4
cols = 9

f = plt.figure(figsize=(2*cols,2*rows))
sub_plot = 1
for i in range(val_x.shape[0]):
    if val_y[i]!=val_p[i]:
        f.add_subplot(rows,cols,sub_plot) 
        sub_plot+=1
        plt.imshow(val_x[i].reshape([28,28]),cmap="Blues")
        plt.axis("off")
        plt.title("T: "+str(val_y[i])+" P:"+str(val_p[i]), y=-0.15,color="Red")
plt.savefig("error_plots.png")
plt.show()

# Predict on Testset

In [ ]:
test_y = np.argmax(model.predict(test_x),axis =1)
# numpy.argmax(array, axis) 用于返回一个numpy数组中最大值的索引值

In [ ]:
model.predict(test_x).shape

In [ ]:
np.argmax(model.predict(test_x),axis =1).shape

In [ ]:
rows = 5
cols = 10

f = plt.figure(figsize=(2*cols,2*rows))

for i in range(rows*cols):
    f.add_subplot(rows,cols,i+1)
    plt.imshow(test_x[i].reshape([28,28]),cmap="Blues")
    plt.axis("off")
    plt.title(str(test_y[i]))

# Lets understand the intermediate layers of the model

<hr>

![hidden layers](https://i.stack.imgur.com/axn7z.jpg)

To visualize the output of each layer, we need to create a model to take input tensor and gives the list of output tensor, each representing the corresponding intermediate layers. To that, we need to create a multi-output model in which the input will be the image, and the output will be the list of intermediate layers.
<br>
为了可视化每一层的输出，我们需要创建一个模型来获取输入张量，并给出输出张量列表，每个张量表示相应的中间层。为此，我们需要创建一个多输出模型，其中输入为图像，输出为中间层列表。
<br>

I will use the Functional API of Keras to do so. When fed an image input, this model returns the values of the layer activations in the original model. 
<br>
我将使用Keras的函数API来做到这一点。当输入图像时，该模型返回原始模型中的层激活值。
<br>
<br>
I am taking all the intermediate layers except the Flatten and Dense layers. 
<br>
我取了所有中间层，除了平坦和密集的层。
<br>

**model.layers** returns the list of layers of the model. Selecting all the layers except the last four layers. Then passing to **models.Model()** as a list of output layers and the input layer of the original model. It will return a new model having the input and output layers of the original model.
<br>
模型层返回模型的层列表。选择除最后四层以外的所有层。然后传给模型。模型（）作为原始模型的输出层和输入层的列表。它将返回一个具有原始模型的输入和输出层的新模型。


**Check out these functions for more info:**
* [Model()](https://keras.io/models/model/)

In [ ]:
# Extracts the outputs of all layers except Flatten and Dense layers
output_layers = [layer.output for layer in model.layers[:-4]]
# Creates a model that will return these outputs, given the model input (This is multi output model)
activation_model = models.Model(inputs=model.input, outputs=output_layers)

In [ ]:
output_layers

In [ ]:
activation_model

## Try to compare the layer 1 output 
<br>

In [ ]:
# predicting the output of each layers
activations_2  = activation_model.predict(val_x[2].reshape([1,28,28,1]))
activations_6  = activation_model.predict(val_x[7].reshape([1,28,28,1]))
first_activation_layer  = activations_2[0]
first_activation_layer.shape

In [ ]:
first_activation_layer6  = activations_6[0]
first_activation_layer6.shape

**activation_2** and **activation_6** are the output of two different images. **first_activation_layer** represents the output of the first layer of the original model. The shape is (1,28,28,32) where 32 represent the number of **channels**. Let's compare the four different channels of the first layer of two different images.

In [ ]:
rows = 4
cols = 2

f = plt.figure(figsize=(2*cols,2*rows))

for i in range(4):
    f.add_subplot(rows,cols,2*i+1)
    plt.imshow(activations_2[0][0,:,:,i].reshape([28,28]),cmap="Blues")
    plt.axis("off") 

    f.add_subplot(rows,cols,2*i+2)
    plt.imshow(activations_6[0][0,:,:,i].reshape([28,28]),cmap="Blues")
    plt.savefig("layer_output_comparision"+str(i)+".png")
    plt.axis("off")

It seems that each channel is trying to depict something in both images, a kind of similar features. This can be concluded that each channel has been trained to find some specific features of the input images.

## Visualizing All Intermediate Activation Layer可视化所有中间激活层

In [ ]:
def plot_layer(layer,i,layer_name = None):
    rows = layer.shape[-1]/16
    cols = 16

    f = plt.figure(figsize=(1*cols,1*rows))
    # plt.imshow(first_activation_layer[0,:,:,:].reshape([14*4,14*16]),cmap="Blues")
    for i in range(layer.shape[-1]):
        f.add_subplot(rows,cols,i+1)
        plt.imshow(layer[0,:,:,i].reshape([layer.shape[1],layer.shape[2]]),cmap="Blues")
        plt.axis("off")
    f.suptitle(layer_name,fontsize=14)
    plt.savefig("intermidiate_layers"+str(i)+".png")
    plt.show()

In [ ]:
# Visualising each layers
for i,layer in enumerate(activation_model.predict(val_x[6].reshape([1,28,28,1]))):
    plot_layer(layer,i,output_layers[i].name)

So, try to understand what's happening inside the hidden layers:
* The first few layers retaining the shape of image but trying to get the very low-level features such as different types edges.
* As we go into the deeper layers, the activations are more abstract and less visually interpretable. These layers are trying to encode high-level features such as corners, angles, selective borders, etc.
* In the last few layers, We can't visually interpret anything; this is because layers are now encoding even more complex features, or we can say more information about the classes.

I hope you understand the basics of CNN after the visualization of intermediate layers even more.

# Creating submisson

In [ ]:
df_submission = pd.DataFrame([df_test.index+1,test_y],["ImageId","Label"]).transpose()
df_submission.to_csv("submission.csv",index=False)
# numpy.transpose()是对矩阵按照所需的要求的转置

In [ ]:
pd.DataFrame([df_test.index+1,test_y],["ImageId","Label"])

In [ ]:
pd.DataFrame([df_test.index+1,test_y],["ImageId","Label"]).transpose()

# About Me
<hr>
I am Tarun Kumar from India.<br>
Software Developer at Toppr.<br>
Hobbyist Artist. <br>
PE Undergrad from IIT ISM Dhanbad. <br>
Deep Learning, Reinforcement Learning, and Data Science. 
<br>
<br>

### Follow me:

* <a href="https://bit.ly/tarungithub"> GitHub</a>
* <a href="https://bit.ly/tarnkr-youtube">Youtube<a/>
* <a href="https://medium.com/@codeeasy"> Medium</a>
* <a href="https://www.linkedin.com/in/tarun-kumar-iit-ism/">Linkedin</a>


<hr>

# Feedback
* **Your feedback is much appreciated**
* **Please UPVOTE if you LIKE this notebook**
* **Comment if you have any doubts or you found any errors in the notebook**